# Notebook for merging cleaned data together

Read data from the csv-files.

In [6]:
import pandas as pd

elec_production = pd.read_csv('.\\data\\electricity-production-FI_2016-2021_hourly.csv', delimiter=';')
elec_consumption = pd.read_csv('.\\data\\electricity-consumption-FI_2016-2021_hourly.csv', delimiter=';')
elec_prices = pd.read_csv('.\\data\\electricity-prices-FI_2016-2021_hourly.csv', delimiter=';')

# make sure that each dataframe has the same number of rows, and columns named Date and Hour
print(elec_production.head(1))
print(elec_production.shape[0])
print(elec_consumption.head(1))
print(elec_consumption.shape[0])
print(elec_prices.head(1))
print(elec_prices.shape[0])


         Date  Hour  PRODUCTION (MWh)
0  2016-01-01     0            7964.0
49685
         Date  Hour  CONSUMP (MWh)
0  2016-01-01     0        10005.0
49685
         Date  Hour PRICE (EUR/MWh)
0  2016-01-01     0           16,39
49685


Merge data together by using datetime information.

In [9]:
from functools import reduce

# here is a list of dataframes that needs to be combined
data_frames = [
    elec_production,
    elec_consumption,
    elec_prices,
]

# merge the frames by usind Date and Hour columns
df = reduce(lambda  left,right: pd.merge(left,right,on=['Date', 'Hour'], how='outer'), data_frames)

print(df.head(1))

         Date  Hour  PRODUCTION (MWh)  CONSUMP (MWh) PRICE (EUR/MWh)
0  2016-01-01     0            7964.0        10005.0           16,39


Write the merged data into csv-file.

In [10]:
df.to_csv('data_2016-2021_hourly.csv', sep=';', encoding='utf-8', index=False)
print('Success')

Success
